# EXPLORING A YELP DATASET

All my code is from the project at the end of the "[SQL for Data Science](https://www.coursera.org/learn/sql-for-data-science)" course by UC Davis.
The issue to share this code on Github or Kaggle is that all the queries were ran directly on a webpage, with no way to download the database.
Their database is a truncated way of a [downloadable Yelp database](https://www.yelp.com/dataset/download).

This dataset is 9GB of Json files.
I decided to truncate this huge dataset to 10k rows per table as well. Maybe I'll try to query the full dataset eventually.
I then transfort the truncated json into a database with the .sqlite extension with DB Browser.
This database is 270MB, apparently too big for a github repo but fine in Kaggle. Also it seems Jupyter Notebooks/Lab can be interactive here on Kaggle, but static on Github.

My 10k trunc of the dataset is not the same as the coursera one, therefore the results are different.

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [2]:
pd.options.display.max_rows = 50 # maximum number of rows to display

In [3]:
%reload_ext sql

%sql sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite

In [4]:
%%sql
SELECT
  COUNT(*) AS nb_null
FROM
  USER
WHERE
  id IS NULL
  OR NAME IS NULL
  OR review_count IS NULL
  OR yelping_since IS NULL
  OR useful IS NULL
  OR funny IS NULL
  OR cool IS NULL
  OR fans IS NULL
  OR average_stars IS NULL
  OR compliment_hot IS NULL
  OR compliment_more IS NULL
  OR compliment_profile IS NULL
  OR compliment_cute IS NULL
  OR compliment_list IS NULL
  OR compliment_note IS NULL
  OR compliment_plain IS NULL
  OR compliment_cool IS NULL
  OR compliment_funny IS NULL
  OR compliment_writer IS NULL
  OR compliment_photos IS NULL

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


nb_null
0


We can see than none have null values

In [5]:
%%sql
SELECT
  city,
  SUM(review_count) tot_nb_reviews
FROM
  business
GROUP BY
  city
ORDER BY
  tot_nb_reviews DESC
LIMIT
  10

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


city,tot_nb_reviews
Philadelphia,62134
New Orleans,51780
Nashville,32161
Tampa,29893
Tucson,26366
Reno,25826
Indianapolis,23151
Santa Barbara,19463
Saint Louis,14309
Edmonton,6830


In [6]:
%%sql
SELECT
  name,
  review_count
FROM
  USER
ORDER BY
  review_count DESC
LIMIT
  3

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


name,review_count
Bruce,16567
Kim,9941
Nijole,8363


*8. Does posting more reviews correlate with more fans?*

Hard to tell, we would need to calculate the correlation factor between the two columns. I don't know how to do this with SQL alone.

*Please explain your findings and interpretation of the results:*

In [7]:
%%sql
SELECT
  name,
  review_count,
  fans
FROM
  USER
ORDER BY
  review_count DESC
LIMIT
  15

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


name,review_count,fans
Bruce,16567,867
Kim,9941,825
Nijole,8363,921
George,7738,288
Jennifer,6679,828
Sunil,6459,186
Eric,5887,444
Ed,5800,2251
Rob,5511,445
Misha,5163,197


If we order my most fans

In [8]:
%%sql
SELECT
  name,
  review_count,
  fans
FROM
  USER
ORDER BY
  fans DESC
LIMIT
  15

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


name,review_count,fans
Mike,1882,12497
Daniel,4333,3138
Jessica,2101,2627
Peter,3193,2388
Ed,5800,2251
Brittany,2492,2086
Farrah,2073,2073
Stephanie,2000,1761
Kimi,1761,1620
Helina,1645,1570


summary

In [9]:
%%sql
SELECT
  name,
  review_count,
  fans,
  fans / review_count AS nb_fans_per_review
FROM
  USER
ORDER BY
  nb_fans_per_review DESC
LIMIT
  15

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


name,review_count,fans,nb_fans_per_review
Mike,1882,12497,6
Michael,3,6,2
Mark,1,2,2
Chef Frisco,4,8,2
Amanda,3,7,2
Jess,1,2,2
chris,1,2,2
Jane,1221,1357,1
Jessica,2101,2627,1
Farrah,2073,2073,1


Something I was trying to attempt is to divide users into 4 quartiles based on their review_count, and then compute the average or median number of fans for each quartile.

This is a more basic way to visualize a correlation.

I wanted to use the NTILE function but it doesn't appear to be supported by SQLite.

In [10]:
%%sql
SELECT
  name,
  review_count,
  fans,
  NTILE (4) OVER (
    ORDER BY
      review_count
  ) review_count_quartiles
FROM
  USER
ORDER BY
  review_count
LIMIT
  10

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


name,review_count,fans,review_count_quartiles
charles,1,0,1
Meg,1,0,1
Paul,1,0,1
Jeremy,1,0,1
Andrew,1,1,1
Kevin,1,0,1
Brandon,1,0,1
William,1,0,1
Mark,1,2,1
James,1,0,1


-- MANUAL GROUP BY
-- I started looking for the four quartiles manually but the distribution was so stacked to the left it didn't makle much sense.
--First quartiles was a review count of 0 or 1. So I made up some thresholds instead :
-- Users in the top 100 (top 1%), the top 1000 (10%), top 5000 (top half) and bottom 5000 (bottom half)

total number of users : 10000

TOP 100 : WHERE review_count >= 344								avg nb of fans : 56.96
TOP 1000 : WHERE review_count >= 49 AND review_count < 344		avg nb of fans : 7.41
TOP 5000 : WHERE review_count >= 6 AND review_count < 49		avg nb of fans : 0.59
BOTTOM 5000 : WHERE review_count < 6							avg nb of fans : 0.06

-- I found each threshold with a simple count() query.
-- Result : There does seem to be a positive correlation if we group users in this way.

-- 9. Are there more reviews with the word "love" or with the word "hate" in them?

In [11]:
%%sql
SELECT
  SUM(
    CASE
      WHEN text LIKE '%love%' THEN 1
      ELSE 0
    END
  ) AS love_count,
  SUM(
    CASE
      WHEN text LIKE '%hate%' THEN 1
      ELSE 0
    END
  ) AS hate_count
FROM
  review

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


love_count,hate_count
1887,210


Answer: love

10. Find the top 10 users with the most fans:

In [12]:
%%sql
SELECT
  name,
  fans
FROM
  USER
ORDER BY
  fans DESC
LIMIT
  10

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


name,fans
Mike,12497
Daniel,3138
Jessica,2627
Peter,2388
Ed,2251
Brittany,2086
Farrah,2073
Stephanie,1761
Kimi,1620
Helina,1570


# Part 2: Inferences and Analysis

-- 1. Pick one city and category of your choice and group the businesses in that city or category by their overall star rating.
--Compare the businesses with 2-3 stars to the businesses with 4-5 stars and answer the following questions. Include your code.
	
-- i. Do the two groups you chose to analyze have a different distribution of hours?

-- What do they mean by distribution of hours ?

-- We have two groups, seven days, and different types of opening hours for each day and business. What is meant by distribution of hours exactly ?

-- I can create a weekly hours thing with the 168 hours
-- then do so some sort of histogram
-- Final table : mediocre vs high.
-- Rows : all 168h but could hide the ones with 0%
-- Share of mediocre restos open at that time vs share for high.

-- It's probably more advanced that what they're asking
-- ONCE we (inner) join the business and hours table, we're only left with 743 entries for the whole country.
-- There are 10k unique ids in the business table, and 1562 in the hours table, 10k overall.
-- Once we join them we're left with
-- - 743 rows
-- - 117 businesses
-- This is problematic as the sample size is getting smaller, and will likely be too small once we focus on a specific city

-- IF I filter for las vegas, one of the cities with the most businessess in the business table, I'm left with only 13 establishments.

--ENTIRE QUERY (click to expand)

In [13]:
%%sql

SELECT 
    cat_rating,
    COUNT(*) as nb_of_businesses,
    ROUND(AVG(week_total_open_hours),1) as avg_nb_weekly_opening_hours
    FROM
    (
    SELECT
        hours,
        SUM(nb_open_hours) as week_total_open_hours,
        name,
        cat_rating

    FROM
    (
        SELECT --GET TOTAL HOURS OPEN PER WEEK FOR EACH BUSINESS
            hours,
            day,
            TIME(opens_at) as time_open,
            TIME(closes_at) as time_closed,
            ROUND(
                CASE WHEN TIME(closes_at) <= TIME(opens_at) THEN
                cast((julianday(closes_at) - julianday(opens_at) + 1) * 24 as float)
                ELSE
                cast((julianday(closes_at) - julianday(opens_at)) * 24 as float)
                END,1) AS nb_open_hours,
            name,
            cat_rating,
            id,
            business_id
        FROM
        (
        SELECT
                    day,
                    opens_at_v0,
                    CASE
                        WHEN substr(opens_at_v0, -10) LIKE '%-%' THEN substr(opens_at_v0,1,5)
                        ELSE substr(opens_at_v0, 1)
                        END AS opens_at,
                    closes_at,
                    TIME(closes_at) as time_close,
                    name,
                    cat_rating,
                    hours, id, business_id
                FROM
                    (
                SELECT --group establishments into High, Mediocre and Other based on their ratings
                    id,
                    name,
                    --stars,
                    CASE
                    WHEN stars BETWEEN 2 AND 3 THEN 'Mediocre'
                    WHEN stars BETWEEN 4 AND 5 THEN 'High'
                    ELSE 'Other'
                    END AS cat_rating
                FROM business
                --WHERE city = 'Las Vegas'
                ) b
                JOIN 
                (
                SELECT --extract day and time from a day-time cell for opening hours
                    business_id,
                    hours,
                    substr(hours, 1, 3) as day,
                    CASE
                        WHEN substr(hours, -9)  LIKE '%|%' THEN '0' || substr(hours, -8,4)
                        WHEN substr(hours, -10) LIKE '%|%' THEN '0' || substr(hours, -9,4)
                        WHEN substr(hours, -11) LIKE '%|%' AND substr(hours, -5) NOT LIKE '%-%' THEN '0' || substr(hours, -10,4)
                        WHEN substr(hours, -11) LIKE '%|%' AND substr(hours, -5) LIKE '%-%' THEN substr(hours, -10,5)

                        ELSE substr(hours, -11,5)
                        END AS opens_at_v0,
                    CASE
                        WHEN substr(hours, -5) LIKE '%-%' THEN '0' || substr(hours, -4)
                        ELSE substr(hours, -5)
                        END AS closes_at
                FROM hours
                ) h

                on b.id = h.business_id

                ORDER BY day, CAST(opens_at AS TIME)
        ) 
    )

    GROUP BY id
)

GROUP BY cat_rating

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


cat_rating,nb_of_businesses,avg_nb_weekly_opening_hours
High,4482,41.4
Mediocre,2199,54.6
Other,1811,54.4


iii. Are you able to infer anything from the location data provided between these two groups? Explain.

In [14]:
%%sql

SELECT -- QUERY C : share of High rating businesses within the city, share of Mediocre.
    city,
    count,
    ROUND(100*nb_high/count, 1) as share_high,
    ROUND(100*nb_mediocre/count, 1) as share_mediocre

FROM
(
    SELECT -- QUERY B : number of businesses in the city, average rating, total number of reviews in that city, nb of High businesses, nb of Mediocre ones.
        city,
        COUNT(*) as count,
        ROUND(AVG(review_count),1) as avg_review_count,
        SUM(review_count) as total_reviews,
        ROUND(AVG(stars),1) as avg_rating,
        SUM(CASE WHEN cat_rating = 'High' THEN 1 ELSE 0 END) AS nb_high,
        SUM(CASE WHEN cat_rating = 'Mediocre' THEN 1 ELSE 0 END) AS nb_mediocre
    FROM
    (
        SELECT --QUERY A : assigns a category rating
            name,
            city,
            stars,
                CASE
                WHEN stars BETWEEN 2 AND 3 THEN 'Mediocre'
                WHEN stars BETWEEN 4 AND 5 THEN 'High'
                ELSE NULL
                END AS
            cat_rating,
            review_count
        FROM business
    )
    GROUP BY city
    HAVING count >= 20
)

ORDER BY count desc
LIMIT 20

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


city,count,share_high,share_mediocre
Philadelphia,980,49.0,27.0
Tucson,628,51.0,29.0
Tampa,624,50.0,25.0
Indianapolis,503,50.0,29.0
Nashville,467,53.0,23.0
New Orleans,440,65.0,16.0
Reno,418,58.0,21.0
Edmonton,345,43.0,31.0
Saint Louis,303,46.0,28.0
Santa Barbara,267,72.0,15.0


-- 2. Group business based on the ones that are open and the ones that are closed. What differences can you find between the ones 
-- that are still open and the ones that are closed? List at least two differences and the SQL code you used to arrive at your answer.

-- Not exactely sure if is_open boolean means open or not right now, or permanently closed ?

-- i. Difference 1:
-- ii. Difference 2:

In [15]:
%%sql
SELECT
  is_open,
  COUNT(*) AS COUNT,
  ROUND(AVG(review_count), 1) AS avg_review_count,
  ROUND(AVG(stars), 1) AS avg_rating
FROM
  business
GROUP BY
  is_open

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


is_open,COUNT,avg_review_count,avg_rating
0,2033,39.7,3.5
1,7967,48.8,3.6


# PART III -  My own analysis

3. For this last part of your analysis, you are going to choose the type of analysis you want to conduct on the Yelp dataset and are going 
to prepare the data for analysis.

Ideas for analysis include:
- parsing out keywords and business attributes for sentiment analysis,
- clustering businesses to find commonalities or anomalies between them,
- predicting the overall star rating for a business,
- predicting the number of fans a user will have, and so on.

These are just a few examples to get you started, so feel free to be creative and come up with your own problem you want to solve.
Provide answers, in-line, to all of the following:

i. Indicate the type of analysis you chose to do:
ii. Write 1-2 brief paragraphs on the type of data you will need for your analysis and why you chose that data:
iii. Output of your finished dataset:
iv. Provide the SQL code you used to create your final dataset:

In [16]:
%%sql
SELECT
  SUM(
    CASE
      WHEN text LIKE '%tasty%' THEN 1
      ELSE 0
    END
  ) AS tasty_count,
  SUM(
    CASE
      WHEN text LIKE '%kind%' THEN 1
      ELSE 0
    END
  ) AS kind_count,
  SUM(
    CASE
      WHEN text LIKE '%welcoming%' THEN 1
      ELSE 0
    END
  ) AS welcoming_count,
  SUM(
    CASE
      WHEN text LIKE '%fast%' THEN 1
      ELSE 0
    END
  ) AS fast_count,
  SUM(
    CASE
      WHEN text LIKE '%hostile%' THEN 1
      ELSE 0
    END
  ) AS hostile_count,
  SUM(
    CASE
      WHEN text LIKE '%cold%' THEN 1
      ELSE 0
    END
  ) AS cold_count
FROM
  review;

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


tasty_count,kind_count,welcoming_count,fast_count,hostile_count,cold_count
523,534,99,897,5,329


## 3A - ADDING A COUNTRY COLUMN

--Note : I just realised the dataset wasn't restricted to the US.
--But this is made harder by the fact that there isn't a country column, just city and state.
-- That's quite unfortunate.

-- There's on in Buenos Aires, and cities in scotland, germany

In [17]:
%%sql
SELECT
  name,
  latitude,
  longitude,
  state,
  city,
  postal_code
FROM
  business
WHERE
  longitude > -65 --too east to be the US
  AND latitude < 25 --too south
GROUP BY
  state
ORDER BY
  state

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


name,latitude,longitude,state,city,postal_code


-- In the whole business, there are 10k entries, but only 23 distinct states

-- Let's investigate :

In [18]:
%%sql
SELECT
  state,
  AVG(latitude),
  AVG(longitude),
  COUNT(id) AS nb_of_businesses
FROM
  business
GROUP BY
  state
ORDER BY
  state

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


state,AVG(latitude),AVG(longitude),nb_of_businesses
AB,53.53321110386228,-113.50501101601797,395
AZ,32.247462151655405,-110.93462610641997,666
CA,34.438266876722906,-119.72145021038212,358
CO,32.0948541,-110.7734914,1
DE,39.737377786518124,-75.59406686966041,149
FL,27.98438548634534,-82.5664930218129,1727
ID,43.61950376009498,-116.27551029220386,259
IL,38.64109143099342,-90.02010836489934,152
IN,39.81941494063171,-86.15799086586343,741
LA,29.961006101460324,-90.10465055870428,678


For the US, business in only eight states are present :
- AZ	Arizona
- IL	Illinois
- NC	North Carolina
- NV	Nevada
- NY	New York (Only two businesses, with a third one actually being located in vegas)
- OH	Ohio
- PA	Pennsylvania
- SC	South Carolina
- WI	Wisconsin

-- We can see that the dataset isn't perfect, for example :

In [19]:
%%sql
SELECT
  *
FROM
  business
WHERE
  state = 'NY'

 * sqlite:////kaggle/input/yelp-dataset-truncated-to-10k-rows/yelp_10k.sqlite
Done.


id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open
